# Books Recommender System

![](http://labs.criteo.com/wp-content/uploads/2017/08/CustomersWhoBought3.jpg)

This is the second part of my project on Book Data Analysis and Recommendation Systems. 

In my first notebook ([The Story of Book](https://www.kaggle.com/omarzaghlol/goodreads-1-the-story-of-book/)), I attempted at narrating the story of book by performing an extensive exploratory data analysis on Books Metadata collected from Goodreads.

In this notebook, I will attempt at implementing a few recommendation algorithms (Basic Recommender, Content-based and Collaborative Filtering) and try to build an ensemble of these models to come up with our final recommendation system.

# What's in this kernel?

- [Importing Libraries and Loading Our Data](#1)
- [Clean the dataset](#2)
- [Simple Recommender](#3)
    - [Top Books](#4)
    - [Top "Genres" Books](#5)
- [Content Based Recommender](#6)
    - [Cosine Similarity](#7)
    - [Popularity and Ratings](#8)
- [Collaborative Filtering](#9)
    - [User Based](#10)
    - [Item Based](#11)
- [Hybrid Recommender](#12)
- [Conclusion](#13)
- [Save Model](#14)

# Importing Libraries and Loading Our Data <a id="1"></a> <br>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
books = pd.read_csv('../input/goodbooks-10k//books.csv')
ratings = pd.read_csv('../input/goodbooks-10k//ratings.csv')
book_tags = pd.read_csv('../input/goodbooks-10k//book_tags.csv')
tags = pd.read_csv('../input/goodbooks-10k//tags.csv')

# Clean the dataset <a id="2"></a> <br>

As with nearly any real-life dataset, we need to do some cleaning first. When exploring the data I noticed that for some combinations of user and book there are multiple ratings, while in theory there should only be one (unless users can rate a book several times). Furthermore, for the collaborative filtering it is better to have more ratings per user. So I decided to remove users who have rated fewer than 3 books.

In [3]:
books['original_publication_year'] = books['original_publication_year'].fillna(-1).apply(lambda x: int(x) if x != -1 else -1)

In [4]:
ratings_rmv_duplicates = ratings.drop_duplicates()
unwanted_users = ratings_rmv_duplicates.groupby('user_id')['user_id'].count()
unwanted_users = unwanted_users[unwanted_users < 3]
unwanted_ratings = ratings_rmv_duplicates[ratings_rmv_duplicates.user_id.isin(unwanted_users.index)]
new_ratings = ratings_rmv_duplicates.drop(unwanted_ratings.index)

In [5]:
new_ratings['title'] = books.set_index('id').title.loc[new_ratings.book_id].values

In [6]:
new_ratings.head(10)

,book_id,user_id,rating,title
0,1,314,5,"The Hunger Games (The Hunger Games, #1)"
1,1,439,3,"The Hunger Games (The Hunger Games, #1)"
2,1,588,5,"The Hunger Games (The Hunger Games, #1)"
3,1,1169,4,"The Hunger Games (The Hunger Games, #1)"
4,1,1185,4,"The Hunger Games (The Hunger Games, #1)"
5,1,2077,4,"The Hunger Games (The Hunger Games, #1)"
6,1,2487,4,"The Hunger Games (The Hunger Games, #1)"
7,1,2900,5,"The Hunger Games (The Hunger Games, #1)"
8,1,3662,4,"The Hunger Games (The Hunger Games, #1)"
9,1,3922,5,"The Hunger Games (The Hunger Games, #1)"


# Simple Recommender <a id="3"></a> <br>

The Simple Recommender offers generalized recommnendations to every user based on book popularity and (sometimes) genre. The basic idea behind this recommender is that books that are more popular and more critically acclaimed will have a higher probability of being liked by the average audience. This model does not give personalized recommendations based on the user.

The implementation of this model is extremely trivial. All we have to do is sort our books based on ratings and popularity and display the top books of our list. As an added step, we can pass in a genre argument to get the top books of a particular genre.


I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of ratings for the book
* *m* is the minimum ratings required to be listed in the chart
* *R* is the average rating of the book
* *C* is the mean rating across the whole report

The next step is to determine an appropriate value for *m*, the minimum ratings required to be listed in the chart. We will use **95th percentile** as our cutoff. In other words, for a book to feature in the charts, it must have more ratings than at least 95% of the books in the list.

I will build our overall Top 250 Chart and will define a function to build charts for a particular genre. Let's begin!

In [7]:
v = books['ratings_count']
m = books['ratings_count'].quantile(0.95)
R = books['average_rating']
C = books['average_rating'].mean()
W = (R*v + C*m) / (v + m)

In [8]:
books['weighted_rating'] = W

In [9]:
qualified  = books.sort_values('weighted_rating', ascending=False).head(250)

## Top Books <a id="4"></a> <br>

In [10]:
qualified[['title', 'authors', 'average_rating', 'weighted_rating']].head(15)

,title,authors,average_rating,weighted_rating
24,Harry Potter and the Deathly Hallows (Harry Po...,"J.K. Rowling, Mary GrandPré",4.61,4.555956
26,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",4.54,4.490428
17,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",4.53,4.485090
23,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",4.53,4.483227
1,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",4.44,4.424365
20,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",4.46,4.419054
30,The Help,Kathryn Stockett,4.45,4.405158
38,"A Game of Thrones (A Song of Ice and Fire, #1)",George R.R. Martin,4.45,4.398759
134,"A Storm of Swords (A Song of Ice and Fire, #3)",George R.R. Martin,4.54,4.396645
421,"Harry Potter Boxset (Harry Potter, #1-7)",J.K. Rowling,4.74,4.391147


We see that J.K. Rowling's **Harry Potter** Books occur at the very top of our chart. The chart also indicates a strong bias of Goodreads Users towards particular genres and authors. 

Let us now construct our function that builds charts for particular genres. For this, we will use relax our default conditions to the **85th** percentile instead of 95. 

## Top "Genres" Books <a id="5"></a> <br>

In [11]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [12]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [13]:
genres = ["Art", "Biography", "Business", "Chick Lit", "Children's", "Christian", "Classics",
          "Comics", "Contemporary", "Cookbooks", "Crime", "Ebooks", "Fantasy", "Fiction",
          "Gay and Lesbian", "Graphic Novels", "Historical Fiction", "History", "Horror",
          "Humor and Comedy", "Manga", "Memoir", "Music", "Mystery", "Nonfiction", "Paranormal",
          "Philosophy", "Poetry", "Psychology", "Religion", "Romance", "Science", "Science Fiction", 
          "Self Help", "Suspense", "Spirituality", "Sports", "Thriller", "Travel", "Young Adult"]

In [14]:
genres = list(map(str.lower, genres))
genres[:4]

['art', 'biography', 'business', 'chick lit']

In [15]:
available_genres = tags.loc[tags.tag_name.str.lower().isin(genres)]

In [16]:
available_genres.head()

,tag_id,tag_name
2938,2938,art
4605,4605,biography
5951,5951,business
7077,7077,christian
7457,7457,classics


In [17]:
available_genres_books = book_tags[book_tags.tag_id.isin(available_genres.tag_id)]

In [18]:
print('There are {} books that are tagged with above genres'.format(available_genres_books.shape[0]))

There are 60573 books that are tagged with above genres


In [19]:
available_genres_books.head()

,goodreads_book_id,tag_id,count
1,1,11305,37174
5,1,11743,9954
25,1,7457,958
38,1,22973,673
52,1,20939,465


In [20]:
available_genres_books['genre'] = available_genres.tag_name.loc[available_genres_books.tag_id].values
available_genres_books.head()

,goodreads_book_id,tag_id,count,genre
1,1,11305,37174,fantasy
5,1,11743,9954,fiction
25,1,7457,958,classics
38,1,22973,673,paranormal
52,1,20939,465,mystery


In [21]:
def build_chart(genre, percentile=0.85):
    df = available_genres_books[available_genres_books['genre'] == genre.lower()]
    qualified = books.set_index('book_id').loc[df.goodreads_book_id]

    v = qualified['ratings_count']
    m = qualified['ratings_count'].quantile(percentile)
    R = qualified['average_rating']
    C = qualified['average_rating'].mean()
    qualified['weighted_rating'] = (R*v + C*m) / (v + m)

    qualified.sort_values('weighted_rating', ascending=False, inplace=True)
    return qualified

Let us see our method in action by displaying the Top 15 Fiction Books (Fiction almost didn't feature at all in our Generic Top Chart despite being one of the most popular movie genres).

In [22]:
cols = ['title','authors','original_publication_year','average_rating','ratings_count','work_text_reviews_count','weighted_rating']

In [23]:
genre = 'Fiction'
build_chart(genre)[cols].head(15)

,title,authors,original_publication_year,average_rating,ratings_count,work_text_reviews_count,weighted_rating
book_id,,,,,,,
136251,Harry Potter and the Deathly Hallows (Harry Po...,"J.K. Rowling, Mary GrandPré",2007,4.61,1746574,51942,4.587098
862041,"Harry Potter Boxset (Harry Potter, #1-7)",J.K. Rowling,1998,4.74,190050,6508,4.544691
1,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",2005,4.54,1678823,27520,4.518933
5,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",1999,4.53,1832823,36099,4.510997
6,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",2000,4.53,1753043,31084,4.510164
62291,"A Storm of Swords (A Song of Ice and Fire, #3)",George R.R. Martin,2000,4.54,469022,19497,4.471466
186074,The Name of the Wind (The Kingkiller Chronicle...,Patrick Rothfuss,2007,4.55,400101,28631,4.469922
1215032,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,2011,4.57,245686,15503,4.446163
18512,"The Return of the King (The Lord of the Rings,...",J.R.R. Tolkien,1955,4.51,463959,6644,4.444645


For simplicity, you can just pass the index of the wanted genre from below. 

In [24]:
list(enumerate(available_genres.tag_name))

[(0, 'art'),
 (1, 'biography'),
 (2, 'business'),
 (3, 'christian'),
 (4, 'classics'),
 (5, 'comics'),
 (6, 'contemporary'),
 (7, 'cookbooks'),
 (8, 'crime'),
 (9, 'ebooks'),
 (10, 'fantasy'),
 (11, 'fiction'),
 (12, 'history'),
 (13, 'horror'),
 (14, 'manga'),
 (15, 'memoir'),
 (16, 'music'),
 (17, 'mystery'),
 (18, 'nonfiction'),
 (19, 'paranormal'),
 (20, 'philosophy'),
 (21, 'poetry'),
 (22, 'psychology'),
 (23, 'religion'),
 (24, 'romance'),
 (25, 'science'),
 (26, 'spirituality'),
 (27, 'sports'),
 (28, 'suspense'),
 (29, 'thriller'),
 (30, 'travel')]

In [25]:
idx = 24  # romance
build_chart(list(available_genres.tag_name)[idx])[cols].head(15)

,title,authors,original_publication_year,average_rating,ratings_count,work_text_reviews_count,weighted_rating
book_id,,,,,,,
136251,Harry Potter and the Deathly Hallows (Harry Po...,"J.K. Rowling, Mary GrandPré",2007,4.61,1746574,51942,4.586504
862041,"Harry Potter Boxset (Harry Potter, #1-7)",J.K. Rowling,1998,4.74,190050,6508,4.540846
1,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",2005,4.54,1678823,27520,4.518389
62291,"A Storm of Swords (A Song of Ice and Fire, #3)",George R.R. Martin,2000,4.54,469022,19497,4.469871
186074,The Name of the Wind (The Kingkiller Chronicle...,Patrick Rothfuss,2007,4.55,400101,28631,4.468099
1215032,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,2011,4.57,245686,15503,4.443591
2,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",2003,4.46,1735368,28685,4.442161
13496,"A Game of Thrones (A Song of Ice and Fire, #1)",George R.R. Martin,1996,4.45,1319204,46205,4.427319
4502507,The Last Olympian (Percy Jackson and the Olymp...,Rick Riordan,2009,4.50,397500,17693,4.425116


# Content Based Recommender <a id="6"></a> <br>

![](https://miro.medium.com/max/828/1*1b-yMSGZ1HfxvHiJCiPV7Q.png)

The recommender we built in the previous section suffers some severe limitations. For one, it gives the same recommendation to everyone, regardless of the user's personal taste. If a person who loves business books (and hates fiction) were to look at our Top 15 Chart, s/he wouldn't probably like most of the books. If s/he were to go one step further and look at our charts by genre, s/he wouldn't still be getting the best recommendations.

For instance, consider a person who loves *The Fault in Our Stars*, *Twilight*. One inference we can obtain is that the person loves the romaintic books. Even if s/he were to access the romance chart, s/he wouldn't find these as the top recommendations.

To personalise our recommendations more, I am going to build an engine that computes similarity between movies based on certain metrics and suggests books that are most similar to a particular book that a user liked. Since we will be using book metadata (or content) to build this engine, this also known as **Content Based Filtering.**

I will build this recommender based on book's *Title*, *Authors* and *Genres*.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

My approach to building the recommender is going to be extremely *hacky*. These are steps I plan to do:
1. **Strip Spaces and Convert to Lowercase** from authors. This way, our engine will not confuse between **Stephen Covey** and **Stephen King**.
2. Combining books with their corresponding **genres** .
2. I then use a **Count Vectorizer** to create our count matrix.

Finally, we calculate the cosine similarities and return books that are most similar.

In [27]:
books['authors'] = books['authors'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x.split(', ')])

In [28]:
def get_genres(x):
    t = book_tags[book_tags.goodreads_book_id==x]
    return [i.lower().replace(" ", "") for i in tags.tag_name.loc[t.tag_id].values]

In [29]:
books['genres'] = books.book_id.apply(get_genres)

In [30]:
books['soup'] = books.apply(lambda x: ' '.join([x['title']] + x['authors'] + x['genres']), axis=1)

In [31]:
books.soup.head()

0    The Hunger Games (The Hunger Games, #1) suzann...
1    Harry Potter and the Sorcerer's Stone (Harry P...
2    Twilight (Twilight, #1) stepheniemeyer young-a...
3    To Kill a Mockingbird harperlee classics favor...
4    The Great Gatsby f.scottfitzgerald classics fa...
Name: soup, dtype: object

In [32]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(books['soup'])

## Cosine Similarity <a id="7"></a> <br>

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two books. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $



In [33]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [34]:
indices = pd.Series(books.index, index=books['title'])
titles = books['title']

In [35]:
def get_recommendations(title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    book_indices = [i[0] for i in sim_scores]
    return list(titles.iloc[book_indices].values)[:n]

In [36]:
get_recommendations("The One Minute Manager")

["Good to Great: Why Some Companies Make the Leap... and Others Don't",
 "First, Break All the Rules: What the World's Greatest Managers Do Differently",
 'Execution: The Discipline of Getting Things Done',
 "What Got You Here Won't Get You There: How Successful People Become Even More Successful",
 'Start with Why: How Great Leaders Inspire Everyone to Take Action',
 'Great by Choice: Uncertainty, Chaos, and Luck--Why Some Thrive Despite Them All',
 'The 21 Irrefutable Laws of Leadership: Follow Them and People Will Follow You',
 'The Speed of Trust: The One Thing that Changes Everything',
 'Fish: A Proven Way to Boost Morale and Improve Results',
 'Leadership and Self-Deception: Getting Out of the Box']

What if I want a specific book but I can't remember it's full name!!

So I created the following *method* to get book titles from a **partial** title.

In [37]:
def get_name_from_partial(title):
    return list(books.title[books.title.str.lower().str.contains(title) == True].values)

In [38]:
title = "business"
l = get_name_from_partial(title)
list(enumerate(l))

[(0, 'The Power of Habit: Why We Do What We Do in Life and Business'),
 (1,
  "The Lean Startup: How Today's Entrepreneurs Use Continuous Innovation to Create Radically Successful Businesses"),
 (2,
  'Caps for Sale: A Tale of a Peddler, Some Monkeys and Their Monkey Business'),
 (3,
  "The E-Myth Revisited: Why Most Small Businesses Don't Work and What to Do About It"),
 (4, 'The Snowball: Warren Buffett and the Business of Life'),
 (5,
  "The Innovator's Dilemma: The Revolutionary Book that Will Change the Way You Do Business (Collins Business Essentials)"),
 (6, 'The Intelligent Investor (Collins Business Essentials)'),
 (7, 'Purple Cow: Transform Your Business by Being Remarkable'),
 (8, 'Business Model Generation'),
 (9, 'The Long Tail: Why the Future of Business is Selling Less of More'),
 (10,
  "Losing My Virginity: How I've Survived, Had Fun, and Made a Fortune Doing Business My Way"),
 (11,
  'The Hard Thing About Hard Things: Building a Business When There Are No Easy Answer

In [39]:
get_recommendations(l[1])

['Rework',
 'The Hard Thing About Hard Things: Building a Business When There Are No Easy Answers',
 'Blue Ocean Strategy: How To Create Uncontested Market Space And Make The Competition Irrelevant',
 'The Art of the Start: The Time-Tested, Battle-Hardened Guide for Anyone Starting Anything',
 "Good to Great: Why Some Companies Make the Leap... and Others Don't",
 'Start with Why: How Great Leaders Inspire Everyone to Take Action',
 'Zero to One: Notes on Startups, or How to Build the Future',
 "The E-Myth Revisited: Why Most Small Businesses Don't Work and What to Do About It",
 'How Google Works',
 'Delivering Happiness: A Path to Profits, Passion, and Purpose']

## Popularity and Ratings <a id="8"></a> <br>

One thing that we notice about our recommendation system is that it recommends books regardless of ratings and popularity. It is true that ***Across the River and Into the Trees*** and ***The Old Man and the Sea*** were written by **Ernest Hemingway**, but the former one was cnosidered a bad (not the worst) book that shouldn't be recommended to anyone, since that most people hated the book for it's static plot and overwrought emotion.

Therefore, we will add a mechanism to remove bad books and return books which are popular and have had a good critical response.

I will take the top 30 movies based on similarity scores and calculate the vote of the 60th percentile book. Then, using this as the value of $m$, we will calculate the weighted rating of each book using IMDB's formula like we did in the Simple Recommender section.

In [40]:
def improved_recommendations(title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    book_indices = [i[0] for i in sim_scores]
    df = books.iloc[book_indices][['title', 'ratings_count', 'average_rating', 'weighted_rating']]

    v = df['ratings_count']
    m = df['ratings_count'].quantile(0.60)
    R = df['average_rating']
    C = df['average_rating'].mean()
    df['weighted_rating'] = (R*v + C*m) / (v + m)
    
    qualified = df[df['ratings_count'] >= m]
    qualified = qualified.sort_values('weighted_rating', ascending=False)
    return qualified.head(n)

In [41]:
improved_recommendations("The One Minute Manager")

,title,ratings_count,average_rating,weighted_rating
2559,The 21 Irrefutable Laws of Leadership: Follow ...,30255,4.12,4.060190
246,The 7 Habits of Highly Effective People: Power...,314700,4.05,4.045478
3234,Start with Why: How Great Leaders Inspire Ever...,32899,4.07,4.035066
931,Good to Great: Why Some Companies Make the Lea...,85277,4.04,4.028535
2326,The Five Dysfunctions of a Team: A Leadership ...,40239,4.01,4.002625
2387,"Delivering Happiness: A Path to Profits, Passi...",37601,4.01,4.002321
2413,The E-Myth Revisited: Why Most Small Businesse...,37671,3.98,3.984657
2219,Built to Last: Successful Habits of Visionary ...,39618,3.98,3.984520
3360,"First, Break All the Rules: What the World's G...",27207,3.92,3.955049
989,Rework,88626,3.93,3.944028


In [42]:
improved_recommendations(l[1])

,title,ratings_count,average_rating,weighted_rating
2165,"Zero to One: Notes on Startups, or How to Buil...",47807,4.17,4.097532
3234,Start with Why: How Great Leaders Inspire Ever...,32899,4.07,4.024009
931,Good to Great: Why Some Companies Make the Lea...,85277,4.04,4.022673
2387,"Delivering Happiness: A Path to Profits, Passi...",37601,4.01,3.992207
2219,Built to Last: Successful Habits of Visionary ...,39618,3.98,3.974785
2413,The E-Myth Revisited: Why Most Small Businesse...,37671,3.98,3.974627
1925,Made to Stick: Why Some Ideas Survive and Othe...,46736,3.97,3.968913
3360,"First, Break All the Rules: What the World's G...",27207,3.92,3.943209
989,Rework,88626,3.93,3.938527
2685,Blue Ocean Strategy: How To Create Uncontested...,30665,3.86,3.909643


I think the sorting of similar is more better now than before.
Therefore, we will conclude our Content Based Recommender section here and come back to it when we build a hybrid engine.


# Collaborative Filtering <a id="9"></a> <br>

![](https://miro.medium.com/max/706/1*DYJ-HQnOVvmm5suNtqV3Jw.png)

Our content based engine suffers from some severe limitations. It is only capable of suggesting books which are *close* to a certain book. That is, it is not capable of capturing tastes and providing recommendations across genres.

Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a book will receive the same recommendations for that book, regardless of who s/he is.

Therefore, in this section, we will use a technique called **Collaborative Filtering** to make recommendations to Book Readers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

I will not be implementing Collaborative Filtering from scratch. Instead, I will use the **Surprise** library that used extremely powerful algorithms like **Singular Value Decomposition (SVD)** to minimise RMSE (Root Mean Square Error) and give great recommendations.

There are two classes of Collaborative Filtering:
![](https://miro.medium.com/max/1280/1*QvhetbRjCr1vryTch_2HZQ.jpeg)
- **User-based**, which measures the similarity between target users and other users.
- **Item-based**, which measures the similarity between the items that target users rate or interact with and other items.

## - User Based <a id="10"></a> <br>

In [43]:
# ! pip install surprise

In [44]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [45]:
reader = Reader()
data = Dataset.load_from_df(new_ratings[['user_id', 'book_id', 'rating']], reader)

In [46]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.83882887, 0.84345675, 0.84527162, 0.83917838, 0.84228921]),
 'test_mae': array([0.65660753, 0.66041893, 0.66080835, 0.65641694, 0.65736882]),
 'fit_time': (99.08930611610413,
  99.54047131538391,
  98.81991958618164,
  101.95654845237732,
  99.66678786277771),
 'test_time': (3.9649055004119873,
  3.9578287601470947,
  3.986180067062378,
  4.304950714111328,
  3.8260459899902344)}

We get a mean **Root Mean Sqaure Error** of about 0.8419 which is more than good enough for our case. Let us now train on our dataset and arrive at predictions.

In [47]:
trainset = data.build_full_trainset()
svd.fit(trainset);

Let us pick users 10 and check the ratings s/he has given.

In [48]:
new_ratings[new_ratings['user_id'] == 10]

,book_id,user_id,rating,title
150478,1506,10,4,The Zahir
282986,2833,10,4,The Prisoner of Heaven (The Cemetery of Forgot...
340448,3409,10,5,The Winner Stands Alone
393966,3946,10,5,Matterhorn
452158,4531,10,4,The Joke
506878,5084,10,2,The Sheltering Sky
588312,5907,10,4,Our Mutual Friend
590191,5926,10,2,The Night Watch
610487,6131,10,2,The Longest Day
696035,7002,10,5,A Mercy


In [49]:
svd.predict(10, 1506)

Prediction(uid=10, iid=1506, r_ui=None, est=3.2689631343421537, details={'was_impossible': False})

For book with ID 1506, we get an estimated prediction of **3.393**. One startling feature of this recommender system is that it doesn't care what the book is (or what it contains). It works purely on the basis of an assigned book ID and tries to predict ratings based on how the other users have predicted the book.

## - Item Based <a id="11"></a> <br>

Here we will build a table for users with their corresponding ratings for each book. 

In [50]:
# bookmat = new_ratings.groupby(['user_id', 'title'])['rating'].mean().unstack()
bookmat = new_ratings.pivot_table(index='user_id', columns='title', values='rating')
bookmat.head()

title,"Angels (Walsh Family, #3)","""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",#GIRLBOSS,'Salem's Lot,"'Tis (Frank McCourt, #2)","1,000 Places to See Before You Die",1/4 جرام,"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","100 Bullets, Vol. 1: First Shot, Last Call",100 Love Sonnets,...,محال,مخطوطة بن إسحاق: مدينة الموتى,نادي السيارات,هشت کتاب,هيبتا,واحة الغروب,يوتوبيا,ڤيرتيجو,キスよりも早く1 [Kisu Yorimo Hayaku 1] (Faster than a Kiss #1),美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon Shinsōban 1]
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
def get_similar(title, mat):
    title_user_ratings = mat[title]
    similar_to_title = mat.corrwith(title_user_ratings)
    corr_title = pd.DataFrame(similar_to_title, columns=['correlation'])
    corr_title.dropna(inplace=True)
    corr_title.sort_values('correlation', ascending=False, inplace=True)
    return corr_title

In [52]:
title = "Twilight (Twilight, #1)"
smlr = get_similar(title, bookmat)

In [53]:
smlr.head(10)

,correlation
title,
god is Not Great: How Religion Poisons Everything,1.0
The Day of the Triffids,1.0
Skipping Christmas,1.0
"Splintered (Splintered, #1)",1.0
Better Homes and Gardens New Cook Book,1.0
"Stolen Songbird (The Malediction Trilogy, #1)",1.0
"Bared to You (Crossfire, #1)",1.0
The Autobiography of Malcolm X,1.0
Balzac and the Little Chinese Seamstress,1.0


Ok, we got similar books, but we need to filter them by their *ratings_count*.

In [54]:
smlr = smlr.join(books.set_index('title')['ratings_count'])
smlr.head()

,correlation,ratings_count
title,,
'Salem's Lot,0.275938,228680
'Salem's Lot,0.275938,72797
11/22/63,0.431331,258464
"13 Little Blue Envelopes (Little Blue Envelope, #1)",-0.500000,66950
1776,0.301511,130293


Get similar books with at least 500k ratings.

In [55]:
smlr[smlr.ratings_count > 5e5].sort_values('correlation', ascending=False).head(10)

,correlation,ratings_count
title,,
"Twilight (Twilight, #1)",1.000000,3866839
"New Moon (Twilight, #2)",0.885400,1149630
"The Selection (The Selection, #1)",0.866025,505340
"Eclipse (Twilight, #3)",0.857845,1134511
"Me Before You (Me Before You, #1)",0.771845,587647
"Matched (Matched, #1)",0.707029,511815
"Breaking Dawn (Twilight, #4)",0.689029,1070245
Bossypants,0.669954,506250
"City of Bones (The Mortal Instruments, #1)",0.654081,1154031


That's more interesting and reasonable result, since we could get *Twilight* book series in our top results. 

# Hybrid Recommender <a id="12"></a> <br>

![](https://www.toonpool.com/user/250/files/hybrid_20095.jpg)

In this section, I will try to build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Book
* **Output:** Similar books sorted on the basis of expected ratings by that particular user.

In [56]:
def hybrid(user_id, title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    book_indices = [i[0] for i in sim_scores]
    
    df = books.iloc[book_indices][['book_id', 'title', 'original_publication_year', 'ratings_count', 'average_rating']]
    df['est'] = df['book_id'].apply(lambda x: svd.predict(user_id, x).est)
    df = df.sort_values('est', ascending=False)
    return df.head(n)

In [57]:
hybrid(4, 'Eat, Pray, Love')

,book_id,title,original_publication_year,ratings_count,average_rating,est
382,1241,A Million Little Pieces,2003,184241,3.62,4.112804
4038,6365221,Mennonite in a Little Black Dress: A Memoir of...,2009,23096,3.17,3.889770
604,40173,"Are You There, Vodka? It's Me, Chelsea",2007,127096,3.85,3.889770
3984,46190,Love Is a Mix Tape,2007,21971,3.83,3.889770
744,12868761,Let's Pretend This Never Happened: A Mostly Tr...,2012,118475,3.90,3.889770
4724,13642929,My Beloved World,2013,17742,4.03,3.889770
5702,316558,Kabul Beauty School: An American Woman Goes Be...,2007,17002,3.63,3.889770
2803,18039963,A House in the Sky,2013,29369,4.20,3.889770
2701,6114607,"The Midwife: A Memoir of Birth, Joy, and Hard ...",2002,19176,4.17,3.889770
616,6314763,Orange Is the New Black,2010,127186,3.70,3.889770


In [58]:
hybrid(10, 'Eat, Pray, Love')

,book_id,title,original_publication_year,ratings_count,average_rating,est
80,7445,The Glass Castle,2005,621099,4.24,3.987087
382,1241,A Million Little Pieces,2003,184241,3.62,3.980582
4038,6365221,Mennonite in a Little Black Dress: A Memoir of...,2009,23096,3.17,3.869284
604,40173,"Are You There, Vodka? It's Me, Chelsea",2007,127096,3.85,3.869284
3984,46190,Love Is a Mix Tape,2007,21971,3.83,3.869284
744,12868761,Let's Pretend This Never Happened: A Mostly Tr...,2012,118475,3.90,3.869284
4724,13642929,My Beloved World,2013,17742,4.03,3.869284
5702,316558,Kabul Beauty School: An American Woman Goes Be...,2007,17002,3.63,3.869284
2803,18039963,A House in the Sky,2013,29369,4.20,3.869284
2701,6114607,"The Midwife: A Memoir of Birth, Joy, and Hard ...",2002,19176,4.17,3.869284


We see that for our hybrid recommender, we get (almost) different recommendations for different users although the book is the same. But maybe we can make it better through following steps:
1. Use our *improved_recommendations* technique , that we used in the **Content Based** seciton above
2. Combine it with the user *estimations*, by dividing their summation by 2
3. Finally, put the result into a new feature ***score***

In [59]:
def improved_hybrid(user_id, title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    book_indices = [i[0] for i in sim_scores]
    
    df = books.iloc[book_indices][['book_id', 'title', 'ratings_count', 'average_rating', 'original_publication_year']]
    v = df['ratings_count']
    m = df['ratings_count'].quantile(0.60)
    R = df['average_rating']
    C = df['average_rating'].mean()
    df['weighted_rating'] = (R*v + C*m) / (v + m)
    
    df['est'] = df['book_id'].apply(lambda x: svd.predict(user_id, x).est)
    
    df['score'] = (df['est'] + df['weighted_rating']) / 2
    df = df.sort_values('score', ascending=False)
    return df[['book_id', 'title', 'original_publication_year', 'ratings_count', 'average_rating', 'score']].head(n)

In [60]:
improved_hybrid(4, 'Eat, Pray, Love')

,book_id,title,original_publication_year,ratings_count,average_rating,score
328,2318271,The Last Lecture,2008,241869,4.25,4.044467
80,7445,The Glass Castle,2005,621099,4.24,4.003138
198,12691,Marley and Me: Life and Love With the World's ...,2005,367304,4.12,3.993449
1669,104189,Same Kind of Different as Me,2005,52964,4.21,3.980154
2803,18039963,A House in the Sky,2013,29369,4.20,3.953617
753,6366437,Half Broke Horses,2008,110597,4.05,3.947693
1067,29209,The Color of Water: A Black Man's Tribute to H...,1996,80906,4.06,3.945557
6286,8564644,Little Princes: One Man's Promise to Bring Hom...,2010,14765,4.25,3.935410
2701,6114607,"The Midwife: A Memoir of Birth, Joy, and Hard ...",2002,19176,4.17,3.932377
4593,31845516,Love Warrior,2016,20094,4.10,3.921842


In [61]:
improved_hybrid(10, 'Eat, Pray, Love')

,book_id,title,original_publication_year,ratings_count,average_rating,score
80,7445,The Glass Castle,2005,621099,4.24,4.103036
328,2318271,The Last Lecture,2008,241869,4.25,4.034225
198,12691,Marley and Me: Life and Love With the World's ...,2005,367304,4.12,3.983206
1669,104189,Same Kind of Different as Me,2005,52964,4.21,3.969911
2803,18039963,A House in the Sky,2013,29369,4.20,3.943375
753,6366437,Half Broke Horses,2008,110597,4.05,3.937450
1067,29209,The Color of Water: A Black Man's Tribute to H...,1996,80906,4.06,3.935314
6286,8564644,Little Princes: One Man's Promise to Bring Hom...,2010,14765,4.25,3.925167
2701,6114607,"The Midwife: A Memoir of Birth, Joy, and Hard ...",2002,19176,4.17,3.922135
4593,31845516,Love Warrior,2016,20094,4.10,3.911599


Ok, we see that the new results make more sense, besides to, the recommendations are more personalized and tailored towards particular users.

# Conclusion <a id="13"></a> <br>

In this notebook, I have built 4 different recommendation engines based on different ideas and algorithms. They are as follows:

1. **Simple Recommender:** This system used overall Goodreads Ratings Count and Rating Averages to build Top Books Charts, in general and for a specific genre. The IMDB Weighted Rating System was used to calculate ratings on which the sorting was finally performed.
2. **Content Based Recommender:** We built content based engines that took book title, authors and genres as input to come up with predictions. We also deviced a simple filter to give greater preference to books with more votes and higher ratings.
3. **Collaborative Filtering:** We built two Collaborative Filters; 
  - one that uses the powerful Surprise Library to build an **user-based** filter based on single value decomposition, since the RMSE obtained was less than 1, and the engine gave estimated ratings for a given user and book.
  - And the other (**item-based**) which built a pivot table for users ratings corresponding to each book, and the engine gave similar books for a given book.
4. **Hybrid Engine:** We brought together ideas from content and collaborative filterting to build an engine that gave book suggestions to a particular user based on the estimated ratings that it had internally calculated for that user.

Previous -> [The Story of Book](https://www.kaggle.com/omarzaghlol/goodreads-1-the-story-of-book/)